In [2]:
from amuse.units import units
from amuse.lab import Particles, new_plummer_gas_model
from amuse.ext.protodisk import ProtoPlanetaryDisk
from amuse.units import nbody_system
from amuse_sink_example import *

In [72]:
class SinkParticles():
    def __init__(self, pos, vel, rsink = 0, num_sinks = 2, names = ['primary_sink', 'secondary_sink']):
        """ 
        if num_sinks > 1, pos and vel should be lists or arrays of vectors
        """ 

        # Set up sink particle(s)
        sinks = Particles(num_sinks)
        sinks.name = names
        sinks.mass = 0 | units.MSun # Sinks are strictly massless
        sinks.radius = rsink | units.AU # this can be either a single value or a list/array of the spatial extent of the sink
        if num_sinks > 1:
            # Make sure we have an iterable, otherwise both sinks are identical in position and velocity which is never true
            assert isinstance(pos,list) or isinstance(pos,np.ndarray), f"Arg `pos` should be of type `list` or `numpy.ndarray`, not {type(pos)}."
            assert isinstance(vel,list) or isinstance(vel,np.ndarray), f"Arg `vel` should be of type `list` or `numpy.ndarray`, not {type(vel)}."

            for i,n in enumerate(names):
                sinks[sinks.name == n].position = pos[i].in_(units.AU)
                sinks[sinks.name == n].velocity = vel[i].in_(units.kms)
        else:
            sinks.position = pos.in_(units.AU)
            sinks.velocity = vel.in_(units.kms)

        self.sinks = sinks
        self.nsinks = num_sinks

    def set_sink(self,rsink = None, vsink = None):
        if self.nsinks == 1:
            if rsink is not None:
                self.sink.position = rsink.in_(units.AU)
            if vsink is not None:
                self.sink.velocity = vsink.in_(units.kms)

        else:
            assert isinstance(rsink,list) or isinstance(rsink,np.ndarray), f"Arg `rsink` should be of type `list` or `numpy.ndarray`, not {type(rsink)}."
            assert isinstance(vsink,list) or isinstance(vsink,np.ndarray), f"Arg `vsink` should be of type `list` or `numpy.ndarray`, not {type(vsink)}."
            for i,n in enumerate(self.sink.name):
                self.sinks[self.sinks.name == n].position = rsink[i].in_(units.AU)
                self.sinks[self.sinks.name == n].velocity = vsink[i].in_(units.kms)
            
    def get_sink(self):
        return self.sink

    def hydro_sink_particles(self, bodies):
        """ 
        bodies is the bodies that are either in the sink or not.
        Function taken from amuse/examples/syllabus/hydro_sink_particles.py
        """

        all_lost_particles = Particles()
        for s in self.sinks:
            xs,ys,zs = s.x,s.y,s.z
            radius_squared = s.radius**2
            insink = bodies.select_array(lambda x,y,z: (x-xs)**2+(y-ys)**2+(z-zs)**2 < radius_squared,['x','y','z'])  
            print(f'Found {len(insink)} bodies in the {s.name}')
            
            if len(insink) == 0:
                continue # no need for further calculations for this sink

            cm = s.position*s.mass
            p = s.velocity*s.mass
            s.mass += insink.total_mass()
            s.position = (cm+insink.center_of_mass()*insink.total_mass())/s.mass
            s.velocity = (p+insink.total_momentum())/s.mass
            all_lost_particles.add_particles(insink)

        return all_lost_particles
    
    def __repr__(self):
        if self.nsinks > 1:
            return f'Sink particles at r = {self.sinks.position} with v = {self.sinks.velocity} in the order (primary, secondary)'
        else:
            return f'Sink particle at r = {self.sinks.position} with v = {self.sinks.velocity}'


In [79]:
from make_system import SystemMaker
from main import *

## Below taken largely from the if name == main part of main.py

# Initialize args from terminal
parser = get_parser()
args = parser.parse_args()

#Adding units to arguments where relevant
smbh_mass = args.m_smbh | units.Msun
outer_semimajor_axis = args.a_out | units.parsec
outer_eccentricity = args.e_out
orbiter_masses = [mass | units.Msun for mass in args.m_orb]
# inner_semimajor_axis = args.a_in | units.AU
inner_semimajor_axis = 0.1 | units.AU
inner_eccentricity = args.e_in
mutual_inclination = args.i_mut | units.deg
arg_of_periapse = args.peri | units.deg
inner_radius = args.r_min | units.AU
outer_radius = args.r_max | units.AU
disk_mass = args.m_disk | units.Msun
diagnostic_timestep = args.dt | units.yr
time_end = args.t_end | units.yr

 #Initializing timesteps
binary_period = orbital_period(sum(args.m_orb) | units.Msun, inner_semimajor_axis) # Assumes circular orbit but works fine 
hydro_timestep = 0.01 * binary_period
gravhydro_timestep = 0.1 * binary_period
print(f'\nHYDRO TIMESTEP: {hydro_timestep.value_in(units.yr):.3f} year, GRAVHYDRO TIMESTEP: {gravhydro_timestep.value_in(units.yr):.3f} year\n')

# Make system, see make_system.py for details
ShaiHulud = SystemMaker(smbh_mass,
                        orbiter_masses,
                        outer_semimajor_axis,
                        outer_eccentricity,
                        inner_semimajor_axis,
                        mutual_inclination,
                        inner_eccentricity,
                        arg_of_periapse,
                        inner_radius,
                        outer_radius,
                        disk_mass,
                        args.n_disk)  # Shai Hulud is the Maker

smbh_and_binary, disk, converter = ShaiHulud.make_system()

#Extract stellar position and velocity from system
pos, vel = [], []
for n in ['primary_star','secondary_star']:
    star = smbh_and_binary[smbh_and_binary.name == n]
    pos.append(star.position.in_(units.AU))
    vel.append(star.velocity.in_(units.kms))

# rsink = (696340 | units.km).in_(units.AU).number #Solar radius
rsink = (10 | units.AU).number
SinkSetter = SinkParticles(pos, vel, rsink = rsink)
print(SinkSetter.sinks)
lost_disk = SinkSetter.hydro_sink_particles(disk)
# print(lost_disk)
print(len(disk))


HYDRO TIMESTEP: 0.000 year, GRAVHYDRO TIMESTEP: 0.002 year

Initializing a binary around a black hole.
                 key         mass         name       radius           vx           vy           vz            x            y            z
                   -         MSun         none           au          kms          kms          kms           au           au           au
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
14400382003822408808    0.000e+00  primary_sink    1.000e+01   -4.434e+01   -8.945e+02   -3.863e+01   -6.171e+03   -1.844e-03    8.283e-03
11460417767234335421    0.000e+00  secondary_sink    1.000e+01    1.701e+02   -9.361e+02    1.482e+02   -6.171e+03    7.072e-03   -3.177e-02
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
Found 627 bodies in the primary_sink
Found 628 bodies in the sec

In [71]:
for s in SinkSetter.sinks:
    print(s.name)

primary_sink
secondary_sink
